In [1]:
import ee
import geemap

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-yankomagn')


Successfully saved authorization token.


In [2]:
# Create a map
Map = geemap.Map()

# Set map center
Map.setCenter(-69.91151172418182, 18.47803423428588, 12)
Map.add_basemap('SATELLITE')

# Load buildings and labels
buildings_uri_SD = '../data/0/overture/santodomingo_buildings.geojson'

In [3]:
# Load buildings
buildings = ee.FeatureCollection('GOOGLE/Research/open-buildings/v3/polygons')
Map.addLayer(buildings, {'color': '00FF00'}, 'Buildings')

In [4]:
# Add labels
labels = ee.FeatureCollection('projects/ee-yankomagn/assets/SD_labels');
labels = labels.map(lambda feature: feature.set('class', 1))
vislabelParams = {'color': 'brown'}
wards = labels.geometry().bounds()

Map.addLayer(wards, {}, 'Wards', False)
Map.addLayer(labels, {}, 'Labels', False)

In [5]:
# Sentinel-2 multispectral imagery collection and processing
def maskS2clouds(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
        qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask) \
        .select(['B4', 'B3', 'B2', 'B8']) \
        .copyProperties(image, ["system:time_start"])

collection = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate('2020-01-01', '2020-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .map(maskS2clouds) \
    .median()

# Clip the image to the study area
image = collection.clip(wards)

# Add the Sentinel-2 layer to the map
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Sentinel-2 RGB', True)

In [6]:
# Calculate building density
empty = ee.Image().byte()
buildingsRec = buildings.map(lambda feature: feature.set({
    'area': ee.Number(1).divide(feature.geometry().area()).multiply(100),
    'const': 1
}))

buildingsRec_color = empty.paint(buildingsRec, 'area').unmask(0)

gaussian = ee.Kernel.gaussian(radius=60, units='meters', normalize=True, sigma=15)

density = buildingsRec_color.reduceNeighborhood(reducer=ee.Reducer.sum(), kernel=gaussian)
clippedDensity = density.clip(wards)
image = image.addBands(clippedDensity)

In [12]:
# Visualization parameters for density
palette = ['blue', 'cyan', 'green', 'yellow', 'red']
visParams = {
    'bands': ['constant_sum'],
    'palette': palette
}

Map.addLayer(image, visParams, 'Density', True)
Map.addLayer(labels, vislabelParams, 'Labels')

In [13]:
# Create a binary mask from the labels
label_mask = ee.Image().paint(labels, 1).unmask(0)

# Function to generate points
def generate_points(region, n_points, label):
    return ee.FeatureCollection.randomPoints(region, n_points) \
        .map(lambda f: f.set('class', label))

# Generate points inside and outside labeled areas
n_points = 100000
points_inside = generate_points(labels.geometry(), n_points, 1)

# Create a small buffer around the labels and use it for the difference operation
labels_buffer = labels.geometry().buffer(1)  # 1 meter buffer
points_outside = generate_points(wards.difference(labels_buffer, 1), n_points, 0)

all_points = points_inside.merge(points_outside)

# Split into training and validation
split = 0.7
training = all_points.randomColumn().filter(ee.Filter.lt('random', split))
validation = all_points.randomColumn().filter(ee.Filter.gte('random', split))

# Sample the image at the point locations
training = image.sampleRegions(
    collection=training,
    properties=['class'],
    scale=10
)

# Train a Random Forest classifier
rf = ee.Classifier.smileRandomForest(100).train(
    features=training,
    classProperty='class',
    inputProperties=['B4', 'B3', 'B2', 'B8', 'constant_sum']
)

# Apply the RF classifier to the image
classified = image.classify(rf)

# Validate the classifier
validated = image.sampleRegions(
    collection=validation,
    properties=['class'],
    scale=10
)
testAccuracy = validated.classify(rf).errorMatrix('class', 'classification')

print('Validation error matrix: ', testAccuracy.getInfo())
print('Validation overall accuracy: ', testAccuracy.accuracy().getInfo())

In [ ]:
# Visualize the classification result
class_palette = ['black', 'red']  # black for background, red for labeled areas
class_vis = {
    'min': 0,
    'max': 1,
    'palette': class_palette
}
Map.addLayer(classified, class_vis, 'Classification')

# Display the map
Map